In [1]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=b3dd22dfd7bb6a900a404851a6d4f71852ea51484424a4bc2569481ab5497360
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from operator import add, abs


spark=SparkSession.builder.config("spark.driver.memory", "15g").appName("iic2440").getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=iic2440>

going to assume that nodes always start on 1 and don't skip numbers

In [3]:
nodes = [1,2,3,4]
edges = [(1,2), (2, 3), (2, 4), (3,2)]

In [4]:
def receiver_value(row):
  buf = row[1]
  receivers = buf[0]
  value = buf[1]
  nnodes = len(receivers)
  value = value/nnodes
  for node in receivers:
    yield (node, value)

#assumes it exist a sparkContect
# damping based form https://en.wikipedia.org/wiki/PageRank
def page_rank(nodes, edges, damping=0.85, epsilon=0.05, MAXITER=10):
  pr = [ (nodes[x], 1/len(nodes)) for x in range(len(nodes))]
  rdd_pr = sc.parallelize(pr)
  rdd_edges = sc.parallelize(edges)
  nnodes = rdd_pr.count()
  delta = epsilon + 1
  extraValue = ((1-damping) / nnodes)
  i = 0
  while epsilon < delta and i < MAXITER:
    new_ranks = rdd_edges.groupByKey().join(rdd_pr).flatMap(receiver_value).reduceByKey(add).mapValues(lambda rank: (rank * damping) + extraValue)
    new_pr = rdd_pr.leftOuterJoin(new_ranks).mapValues(lambda x: x[1] if x[1] is not None else extraValue)
    delta = new_pr.join(rdd_pr).map(lambda x: x[1][1] - x[1][0]).map(abs).sum()
    rdd_pr = new_pr
    i += 1
    print('iter', i, ' | delta', delta)
  return rdd_pr.collect()

def page_rank_non_optimized(nodes, edges, damping=0.85, epsilon=0.05, MAXITER=10):
  pr = [ (nodes[x], 1/len(nodes)) for x in range(len(nodes))]
  rdd_pr = sc.parallelize(pr)
  rdd_edges = sc.parallelize(edges)
  nnodes = rdd_pr.count()
  delta = epsilon + 1
  extraValue = ((1-damping) / nnodes)
  i = 0
  while epsilon < delta and i < MAXITER:
    rdd_node_edge = rdd_edges.groupByKey()
    rdd_msgs = rdd_node_edge.join(rdd_pr)
    rdd_msgs = rdd_msgs.flatMap(receiver_value)
    new_ranks = rdd_msgs.reduceByKey(add).mapValues(lambda rank: (rank * damping) + extraValue)
    new_pr = rdd_pr.leftOuterJoin(new_ranks).mapValues(lambda x: x[1] if x[1] is not None else extraValue)
    delta = new_pr.join(rdd_pr).map(lambda x: x[1][1] - x[1][0]).map(abs).sum()
    rdd_pr = new_pr
    i += 1
    print('iter', i, ' | delta', delta)
  return rdd_pr.collect()

In [5]:
page_rank(nodes, edges)

iter 1  | delta 0.6375
iter 2  | delta 0.4515625000000001
iter 3  | delta 0.3070625
iter 4  | delta 0.16312695312500003
iter 5  | delta 0.110926328125
iter 6  | delta 0.05892961181640627
iter 7  | delta 0.0400721360351563


[(1, 0.037500000000000006),
 (2, 0.17284380207519534),
 (3, 0.10670095142822265),
 (4, 0.10670095142822265)]